In [115]:
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [116]:
user = pd.read_csv('../Data/whyout_data/user.csv') # (31176,3)
drop_user_place_idx = pd.read_csv('../Data/whyout_data/drop_user_place_idx.csv') # (22420,3)
drop_user_product_idx = pd.read_csv('../Data/whyout_data/drop_user_product_idx.csv') # (2296,3)
drop_user_video_idx = pd.read_csv('../Data/whyout_data/drop_user_video_idx.csv') # (11067,3)
common_drop_user_idx = pd.read_csv('../Data/whyout_data/common_drop_user_idx.csv') # (25827,3)

In [117]:
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10)
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5834,11)
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9)

In [114]:
user_place = pd.read_csv('../Data/whyout_data/user_place.csv') # (31176,4697)
user_product = pd.read_csv('../Data/whyout_data/user_product.csv') # (31176,5834)
user_video = pd.read_csv('../Data/whyout_data/user_video.csv') # (31176, 3250)

In [118]:
drop_user_place = pd.read_csv('../Data/whyout_data/drop_user_place.csv') # (22420,4697) 
drop_user_product = pd.read_csv('../Data/whyout_data/drop_user_product.csv') # (2996,5834)
drop_user_video = pd.read_csv('../Data/whyout_data/drop_user_video.csv') # (11067, 3250)

In [119]:
common_drop_user_place = pd.read_csv('../Data/whyout_data/common_drop_user_place.csv') # (25827,4697)
common_drop_user_product = pd.read_csv('../Data/whyout_data/common_drop_user_product.csv') # (25827,5834)
common_drop_user_video = pd.read_csv('../Data/whyout_data/common_drop_user_video.csv') # (25827, 3250)

In [120]:
class SGD():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose
        self.cost_list = []


    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """
        # init latent features
        self._U = np.random.normal(size=(self._num_users, self._k))
        self._V = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_U = np.zeros(self._num_users)
        self._b_V = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):
            # rating이 존재하는 index를 기준으로 training
            xi, yi = self._R.nonzero()
            for i, j in zip(xi, yi):
                self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                self.cost_list.append(cost)
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))
        return self.cost_list


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """
        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        xi, yi = self._R.nonzero()
        # predicted = self.get_complete_matrix()
        cost = 0
        #print(len(xi), len(yi))
        count = 0
        for x, y in zip(xi, yi):
            count += 1
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
            # if self._R[x,y]== 6:
            #     print(cost, self._R[x,y], self.get_prediction(x,y))
        return np.sqrt(cost/len(xi))


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD
        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """
        du = (error * self._V[j, :]) - (self._reg_param * self._U[i, :]) # user에 대해 gradient -> item에 대해 미분
        dv = (error * self._U[i, :]) - (self._reg_param * self._V[j, :])
        return du, dv


    def gradient_descent(self, i, j, rating):
        """
        graident descent function
        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """
        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_U[i] += self._learning_rate * (error - self._reg_param * self._b_U[i])
        self._b_V[j] += self._learning_rate * (error - self._reg_param * self._b_V[j])

        # update latent feature
        du, dv = self.gradient(error, i, j)
        self._U[i, :] += self._learning_rate * du
        self._V[j, :] += self._learning_rate * dv


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_U[i] + self._b_V[j] + self._U[i, :].dot(self._V[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix UXV + U.bias + V.bias + global bias

        - UXV 행렬에 b_U[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_V[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_U[:, np.newaxis] + self._b_V[np.newaxis:, ] + self._U.dot(self._V.T)
    
    def print_results(self):
        print("User Latent U:")
        print(self._U)
        print("Item Latent V:")
        print(self._V.T)
        print("U x V:")
        print(self._U.dot(self._V.T))
        print("bias:")
        print(self._b)
        print("User Latent bias:")
        print(self._b_U)
        print("Item Latent bias:")
        print(self._b_V)
        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][1])

In [121]:
if __name__ == "__main__":
    # rating matrix - User X Item : (사용자 수 X 아이템 수)
    # U, V is (사용자 수 X k), (k X 아이템 수) matrix
    R = np.array(user_place)
    factorizer = SGD(R, k=50, learning_rate=0.01, reg_param=0.01, epochs=1000, verbose=True)
    cost_list = factorizer.fit()
    complete_matrix = factorizer.get_complete_matrix()

    R1 = np.array(user_product)
    factorizer = SGD(R1, k=50, learning_rate=0.01, reg_param=0.01, epochs=1000, verbose=True)
    cost_list1 = factorizer.fit()
    complete_matrix1 = factorizer.get_complete_matrix()

    R2 = np.array(user_video)
    factorizer = SGD(R2, k=50, learning_rate=0.01, reg_param=0.01, epochs=1000, verbose=True)
    cost_list2 = factorizer.fit()
    complete_matrix2 = factorizer.get_complete_matrix()

Iteration: 10 ; cost = 0.7637
Iteration: 20 ; cost = 0.7373
Iteration: 30 ; cost = 0.7177
Iteration: 40 ; cost = 0.6986
Iteration: 50 ; cost = 0.6784
Iteration: 60 ; cost = 0.6567
Iteration: 70 ; cost = 0.6334
Iteration: 80 ; cost = 0.6087
Iteration: 90 ; cost = 0.5831
Iteration: 100 ; cost = 0.5569
Iteration: 110 ; cost = 0.5307
Iteration: 120 ; cost = 0.5048
Iteration: 130 ; cost = 0.4798
Iteration: 140 ; cost = 0.4559
Iteration: 150 ; cost = 0.4334
Iteration: 160 ; cost = 0.4125
Iteration: 170 ; cost = 0.3932
Iteration: 180 ; cost = 0.3755
Iteration: 190 ; cost = 0.3594
Iteration: 200 ; cost = 0.3448
Iteration: 210 ; cost = 0.3316
Iteration: 220 ; cost = 0.3197
Iteration: 230 ; cost = 0.3090
Iteration: 240 ; cost = 0.2994
Iteration: 250 ; cost = 0.2908
Iteration: 260 ; cost = 0.2831
Iteration: 270 ; cost = 0.2764
Iteration: 280 ; cost = 0.2703
Iteration: 290 ; cost = 0.2649
Iteration: 300 ; cost = 0.2600
Iteration: 310 ; cost = 0.2557
Iteration: 320 ; cost = 0.2517
Iteration: 330 ; 

In [ ]:
# 사용자의 평가 유무 확인
def find_zero_indices(df):
    return df.index[df.eq(0).all(axis=1)].tolist()

place_zero_indices = find_zero_indices(user_place)
product_zero_indices = find_zero_indices(user_product)
video_zero_indices = find_zero_indices(user_video)

#common_zero_indices = set(place_zero_indices) & set(product_zero_indices) & set(video_zero_indices)
#place_product_zero_indices = set(place_zero_indices) & set(product_zero_indices)
#place_video_zero_indices = set(place_zero_indices) & set(video_zero_indices)
#product_video_zero_indices = set(product_zero_indices) & set(video_zero_indices)

print('장소를 평가하지 않은 사용자 수:',len(place_zero_indices))
print('상품을 평가하지 않은 사용자 수:',len(product_zero_indices))
print('영상을 평가하지 않은 사용자 수:',len(video_zero_indices))
#print('모두 평가하지 않은 사용자 수:',len(common_zero_indices))
#print('장소 & 상품을 같이 평가하지 않은 사용자 수:',len(place_product_zero_indices))
#print('장소 & 영상을 같이 평가하지 않은 사용자 수:',len(place_video_zero_indices))
#print('상품 & 영상을 같이 평가하지 않은 사용자 수:',len(product_video_zero_indices))

In [122]:
df = pd.DataFrame(complete_matrix).astype(dtype='float16')
df.to_csv('user_place_k50epochs1000.csv', index=False)

df1 = pd.DataFrame(complete_matrix1).astype(dtype='float16')
df1.to_csv('user_product_k50epochs1000.csv', index=False)

df2 = pd.DataFrame(complete_matrix2).astype(dtype='float16')
df2.to_csv('user_video_k50epochs1000.csv', index=False)

In [123]:
if __name__ == "__main__":
    # rating matrix - User X Item : (사용자 수 X 아이템 수)
    # U, V is (사용자 수 X k), (k X 아이템 수) matrix
    R3 = np.array(drop_user_place)
    factorizer = SGD(R3, k=40, learning_rate=0.01, reg_param=0.01, epochs=1000, verbose=True)
    cost_list = factorizer.fit()
    complete_matrix3 = factorizer.get_complete_matrix()

    R4 = np.array(drop_user_product)
    factorizer = SGD(R4, k=20, learning_rate=0.01, reg_param=0.01, epochs=1000, verbose=True)
    cost_list1 = factorizer.fit()
    complete_matrix4 = factorizer.get_complete_matrix()

    R5 = np.array(drop_user_video)
    factorizer = SGD(R5, k=30, learning_rate=0.01, reg_param=0.01, epochs=1000, verbose=True)
    cost_list2 = factorizer.fit()
    complete_matrix5 = factorizer.get_complete_matrix()

Iteration: 10 ; cost = 0.7572
Iteration: 20 ; cost = 0.7342
Iteration: 30 ; cost = 0.7178
Iteration: 40 ; cost = 0.7019
Iteration: 50 ; cost = 0.6851
Iteration: 60 ; cost = 0.6670
Iteration: 70 ; cost = 0.6473
Iteration: 80 ; cost = 0.6262
Iteration: 90 ; cost = 0.6040
Iteration: 100 ; cost = 0.5810
Iteration: 110 ; cost = 0.5578
Iteration: 120 ; cost = 0.5347
Iteration: 130 ; cost = 0.5122
Iteration: 140 ; cost = 0.4905
Iteration: 150 ; cost = 0.4699
Iteration: 160 ; cost = 0.4507
Iteration: 170 ; cost = 0.4328
Iteration: 180 ; cost = 0.4163
Iteration: 190 ; cost = 0.4011
Iteration: 200 ; cost = 0.3873
Iteration: 210 ; cost = 0.3748
Iteration: 220 ; cost = 0.3634
Iteration: 230 ; cost = 0.3532
Iteration: 240 ; cost = 0.3440
Iteration: 250 ; cost = 0.3358
Iteration: 260 ; cost = 0.3284
Iteration: 270 ; cost = 0.3218
Iteration: 280 ; cost = 0.3160
Iteration: 290 ; cost = 0.3107
Iteration: 300 ; cost = 0.3060
Iteration: 310 ; cost = 0.3017
Iteration: 320 ; cost = 0.2978
Iteration: 330 ; 

In [124]:
df3 = pd.DataFrame(complete_matrix3).astype(dtype='float16')
df3.to_csv('drop_user_place_k40epochs1000.csv', index=False)

df4 = pd.DataFrame(complete_matrix4).astype(dtype='float16')
df4.to_csv('drop_user_product_k20epochs1000.csv', index=False)

df5 = pd.DataFrame(complete_matrix5).astype(dtype='float16')
df5.to_csv('drop_user_video_k30epochs1000.csv', index=False)

In [125]:
if __name__ == "__main__":
    # rating matrix - User X Item : (사용자 수 X 아이템 수)
    # U, V is (사용자 수 X k), (k X 아이템 수) matrix
    R6 = np.array(common_drop_user_place)
    factorizer = SGD(R3, k=40, learning_rate=0.01, reg_param=0.01, epochs=1000, verbose=True)
    cost_list = factorizer.fit()
    complete_matrix6 = factorizer.get_complete_matrix()

    R7 = np.array(common_drop_user_product)
    factorizer = SGD(R4, k=40, learning_rate=0.01, reg_param=0.01, epochs=1000, verbose=True)
    cost_list1 = factorizer.fit()
    complete_matrix7 = factorizer.get_complete_matrix()

    R8 = np.array(common_drop_user_video)
    factorizer = SGD(R5, k=40, learning_rate=0.01, reg_param=0.01, epochs=1000, verbose=True)
    cost_list2 = factorizer.fit()
    complete_matrix8 = factorizer.get_complete_matrix()

Iteration: 10 ; cost = 0.7567
Iteration: 20 ; cost = 0.7340
Iteration: 30 ; cost = 0.7176
Iteration: 40 ; cost = 0.7016
Iteration: 50 ; cost = 0.6847
Iteration: 60 ; cost = 0.6664
Iteration: 70 ; cost = 0.6465
Iteration: 80 ; cost = 0.6253
Iteration: 90 ; cost = 0.6029
Iteration: 100 ; cost = 0.5798
Iteration: 110 ; cost = 0.5566
Iteration: 120 ; cost = 0.5336
Iteration: 130 ; cost = 0.5112
Iteration: 140 ; cost = 0.4896
Iteration: 150 ; cost = 0.4692
Iteration: 160 ; cost = 0.4499
Iteration: 170 ; cost = 0.4319
Iteration: 180 ; cost = 0.4153
Iteration: 190 ; cost = 0.4000
Iteration: 200 ; cost = 0.3861
Iteration: 210 ; cost = 0.3736
Iteration: 220 ; cost = 0.3623
Iteration: 230 ; cost = 0.3523
Iteration: 240 ; cost = 0.3433
Iteration: 250 ; cost = 0.3352
Iteration: 260 ; cost = 0.3280
Iteration: 270 ; cost = 0.3216


In [ ]:
df6 = pd.DataFrame(complete_matrix6).astype(dtype='float16')
df6.to_csv('common_drop_user_place_k40epochs1000.csv', index=False)

df7 = pd.DataFrame(complete_matrix7).astype(dtype='float16')
df7.to_csv('common_drop_user_product_k40epochs1000.csv', index=False)

df8 = pd.DataFrame(complete_matrix8).astype(dtype='float16')
df8.to_csv('common_drop_user_video_k40epochs1000.csv', index=False)

In [83]:
# sgd_place_df_full = pd.read_csv('../Data/whyout_data/sgd_result/full_data/place_epochs2000.csv', index_col=0)
# sgd_product_df_full = pd.read_csv('../Data/whyout_data/sgd_result/full_data/product_epochs2000.csv', index_col=0)
# sgd_video_df_full = pd.read_csv('../Data/whyout_data/sgd_result/full_data/video_epochs2000.csv', index_col=0)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [52]:
sgd_place_df = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_place_k50epochs1000.csv')
sgd_product_df = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_product_k20epochs1000.csv')
sgd_video_df = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_video_k40epochs1000.csv')

In [107]:
def recommend_items(df_sgd_preds, user_id, item_df, ratings_df, num_recommendations):
    """
    :param df_sgd_preds: SGD 예측 평점 데이터프레임
    :param user_id: 사용자 ID
    :param df: 아이템 데이터프레임
    :param ratings_df: 사용자 평점 데이터프레임
    :param num_recommendations: 추천할 아이템 수
    :return: 사용자가 이미 평가한 아이템 인덱스와 추천 아이템 데이터프레임
    """
    # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
    user_data = ratings_df.loc[user_id]

    # 사용자가 이미 평가한 상품의 인덱스를 추출
    user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]
    print(len(user_history_indices),user_history_indices)

    # SGD를 통해 예측된 사용자의 상품 평점을 기반으로 상품 데이터를 정렬
    user_row_number = user_id - 1
    sorted_user_predictions = df_sgd_preds.iloc[user_row_number].sort_values(ascending=False)

    # 사용자가 아직 평가하지 않은 상품 목록을 추출
    recommendations = item_df.loc[~item_df.index.isin(user_history_indices)]

    # 예측된 상품 평점 데이터와 원본 상품 데이터를 합침
    predictions_df = pd.DataFrame(sorted_user_predictions).reset_index()
    predictions_df['index'] = predictions_df['index'].astype(int)
    predictions_df.set_index('index', inplace=True)

    # recommendations의 인덱스와 predictions_df의 인덱스를 기준으로 병합
    recommendations = recommendations.merge(predictions_df, left_index=True, right_index=True)
    
    # 컬럼 이름을 바꾸고 정렬해서 반환
    recommendations = recommendations.rename(columns={user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_recommendations]
    print(f"user {user_id}에게 추천해줄 {num_recommendations}개 아이템 id : {recommendations['idx'].values}")
    return user_history_indices, recommendations


In [111]:
already_rated, predictions = recommend_items(sgd_place_df, 2, place, user_place, 10)
# place 10 : user idx 37
# place 7 : user idx 22

ValueError: invalid literal for int() with base 10: '0.7'

In [101]:
predictions

,idx,지역(위치정보),공간유형,주변환경,계절,테마,클릭 수,좋아요 수,북마크 수,댓글 수,Predictions
326,327,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000376,0.0,0.0,0.0,12.440
1956,1957,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",NaN,NaN,NaN,NaN,0.000188,0.0,0.0,0.0,12.240
4110,4131,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000047,0.0,0.0,0.0,11.700
1641,1642,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000094,0.0,0.0,0.0,11.695
1230,1231,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000141,0.0,0.0,0.0,11.630
1551,1552,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000141,0.0,0.0,0.0,11.280
984,985,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000141,0.0,0.0,0.0,11.080
96,97,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000282,0.0,0.0,0.0,10.940
1887,1888,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",NaN,NaN,NaN,NaN,0.000047,0.0,0.0,0.0,10.660
881,882,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000094,0.0,0.0,0.0,10.470
